# <font color='red'>STOCK MARKET IS VERY RISKY UNTIL YOU DO IT PROPERLY. PLEASE DO NOT TAKE TRADES JUST BECAUSE THIS TOOL GIVES YOU THE NAME. APPLY YOUR OWN LEARNINGS, CREATE YOUR OWN STRATEGY, ASSESS RISK & TRADE THE PLAN.</font>

In [1]:
'065676'

'065676'

# Imports

In [2]:
import backtrader
from helpers.datahandler import *
from helpers.investing import *
from helpers.journal_handler import *
from helpers.intraday import *
from helpers.nse_data import *

# Defaults

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

In = Investing(check_fresh = True) # Investing or Swing trading analysis
Intra = IntraDay() # Intraday strategies and analysis
NSE = NSEData() # Get NSE Live data
MS = MarketSentiment() # Get Live Market Sentiment TICK, TRIN and 52W high/ low stocks

Checking Fresh Data.....


# Journal
Make a journal in Google Drive for each trade you take to analyse. It is a not so simple process and need to connect with Google API so if you're not comfortable with it, just skip it

In [4]:
private_key = join(expanduser('~'),'Documents','client_secret.json') # Personal for each user. Mine is stored in /home/Documents
JH = JournalHandler(private_key)

journal = JH.get_journal('Finance Journal') # My Google Sheet jornam has name "Finance Journal"

JH.total_pl(journal) # total Profit and Loss up until now

--------------------------------------------------------------------------- 

Trailing Stop Loss: 

INFY bought on: 2021-09-03 00:00:00,

--------------------------------------------------------------------------- 



/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


638.69

# <font color='teal'>Swing Strategies </font>
## All Details
Basic Details about stocks such as Momentum, Ichimoku, 1-2-3 candles pattern etc

In [5]:
budget = 40000

df = In.get_recent_info(nifty = 100, **{'mvs':[20,50,100,200]})
df = df[(df['Overbought'] == False) & (df['Ichi Count'] > 0) & 
        (df['Over 50-SMA'] == True) & (df['Over 100-SMA'] == True) & (df['Over 200-SMA'] == True) &
       (df['Momentum ADX'] > 25) & 
        (df['LTP'] <= budget)]

df.head(2)

/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/home/admin1/anaconda3/envs/finance/lib/python3.7/site-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,Date,Name,LTP,Index,Over 20-SMA,Over 50-SMA,Over 100-SMA,Over 200-SMA,Overbought,Oversold,Momentum ADX,Ichi Count,1 Candle,2 Candles,3 Candles
6,2021-09-23,ALKEM,3963.2,Nifty 100,True,True,True,True,False,False,33.846144,2,Unknown Green,Unknown,Unknown
8,2021-09-23,APOLLOHOSP,5084.2,Nifty 100,True,True,True,True,False,False,36.677898,2,Unknown Green,Unknown,Unknown


## 44-SMA 
If you find a `Green` candle taking support on 44 days `Simple Moving Average` line and stock is in uptrend, you can take the trade and set Risk to Reward ratio as 1:2

In [6]:
budget = 40000
risk = 400
df = In.show_full_stats(budget=budget,risk=risk, diff = risk, nifty='nifty_50')
df = df[(df['Expected Change %'] <= 15) & (df['RSI']<70) & (df['Direction'] != 'Falling Low')]
df

,DATE,OPEN,HIGH,LOW,CLOSE,52W H,52W L,SYMBOL,Diff,Direction,Ichi,RSI,ATR,Triple Candle,Double Candle,Recent Candle,Index,Expected Change %,Max Config Risk
3,2021-09-23,711.00,720.00,707.15,717.00,734.80,333.75,ICICIBANK,5.52,Rising High,0,55.180642,12.582143,Unknown,Unknown,Unknown Green,Nifty 50,5.32,383.45
22,2021-09-23,117.00,118.65,116.80,118.00,119.45,71.65,IOC,8.26,Rising High,1,67.213608,2.385714,Unknown,Unknown,Unknown Green,Nifty 50,6.45,398.70
0,2021-09-23,442.40,451.15,442.05,449.80,471.90,175.50,SBIN,11.13,Rising High,0,59.924055,10.175000,Unknown,Unknown,Unknown Green,Nifty 50,7.05,398.43
25,2021-09-23,313.20,324.00,313.20,321.50,360.75,122.15,TATAMOTORS,15.96,Undeterministic,1,66.338167,8.307143,Unknown,Unknown,Unknown Green,Nifty 50,13.33,389.29
20,2021-09-23,3757.15,3820.00,3757.15,3799.20,4361.40,2822.45,BAJAJ-AUTO,17.89,Undeterministic,1,53.103840,62.317857,Unknown,Unknown,Unknown Green,Nifty 50,5.51,316.48
18,2021-09-23,1542.00,1572.00,1542.00,1570.00,1641.00,1025.00,HDFCBANK,22.42,Rising High,0,56.220290,25.628571,Unknown,Unknown,Bullish,Nifty 50,5.73,360.82
23,2021-09-23,473.50,484.80,468.00,483.25,488.00,154.40,HINDALCO,23.99,Rising High,0,60.520944,14.560714,Unknown,Unknown,Unknown Green,Nifty 50,13.81,368.91
10,2021-09-23,794.35,814.50,792.55,812.15,820.70,400.15,AXISBANK,26.84,Rising High,0,63.435085,16.350000,Unknown,Unknown,Bullish,Nifty 50,7.38,391.31
12,2021-09-23,965.45,970.00,959.50,968.10,997.00,706.50,CIPLA,30.15,Rising High,1,61.275803,17.325000,Unknown,Unknown,Unknown Green,Nifty 50,4.17,385.16
5,2021-09-23,754.40,760.00,750.55,754.50,901.00,312.10,ADANIPORTS,33.15,Undeterministic,0,57.510978,17.385714,Unknown,Unknown,Inverted Hammer,Nifty 50,5.12,390.04


## Golden Crossover
When 44 days `SMA` Crosses 200 days `SMA` from below and you find a `Green` candle within 15 days taking support on `44` days line set Risk to Reward ratio as 1:5

In [7]:
for name in In.data['nifty_500']:
    df = In.open_downloaded_stock(name)
    if In.has_golden_crossover(df,lookback=10):
        print(name)

ESCORTS
ITC


## Oversold / Overbought Stocks
Buy when RSI is less than 30 sell when greater than 70

In [8]:
for name in In.data['nifty_100']:
    df = In.open_downloaded_stock(name)
    if In.get_RSI(df) < 30:
        print(name)

# <font color='teal'>Intraday Strategies</font>

## Narrow Range 7
If recent candle's range is lowest among 6 other days then the future candle might break it's low/high

In [9]:
for name in In.data['nifty_100']:
    if Intra.NR7_strategy(name):
        print(name)

ABBOTINDIA
ADANIPORTS
ASIANPAINT
AUROPHARMA
EICHERMOT
GODREJCP
HINDPETRO
IGL
INDUSINDBK
NAUKRI
INFY
SHREECEM
TATACONSUM
TECHM
UPL
VEDL
WIPRO


## Whole Number Open
When Open == Low / High in Whole number as `xxx.00` within 15 minutes

In [10]:
result = Intra.whole_number_strategy(nifty=200, min_val=101,max_val=4999)
for key in result.keys():
    print(key,":\n")
    for r in sorted(result[key],key=lambda x: int(x[-1].split(' ')[-1])):
        print(r,'\n')

Long :

('GRASIM', '2.66%', 'Nifty 50') 

('DABUR', '1.53%', 'Nifty 100') 

Short :

('ITC', '-2.58%', 'Nifty 50') 

('VEDL', '-1.87%', 'Nifty 100') 

('NATCOPHARM', '-1.79%', 'Nifty 200') 

('AMARAJABAT', '-1.57%', 'Nifty 200') 

('LALPATHLAB', '-3.7%', 'Nifty 200') 



## `%` Change probability
Gives the stock names which have the highest probability of providing you `x%` in intraday atleast on both long or short side based on the previous `N` days record

In [11]:
# Gives you top 10 stocks from Nifty 500 which have the highest probability of giving atleast 0.99% on your investment based on the Long/Buy position. Data is based on past 60 trading days
# Test it for different time duration, index, % change and Long / Short criteria based on overall market trend

Intra.prob_by_percent_change(symbol = None, index = 500, time_period = 21, change_percent=0.099, sort_by='Long Probability', top_k = 10) 

{'CANFINHOME': {'Long Probability': 0.95, 'Short Probability': 0.43},
 'BRIGADE': {'Long Probability': 0.9, 'Short Probability': 0.71},
 'GUJALKALI': {'Long Probability': 0.9, 'Short Probability': 0.33},
 'LINDEINDIA': {'Long Probability': 0.9, 'Short Probability': 0.71},
 'SUNTECK': {'Long Probability': 0.9, 'Short Probability': 0.67},
 'ATGL': {'Long Probability': 0.86, 'Short Probability': 0.76},
 'CHALET': {'Long Probability': 0.86, 'Short Probability': 0.52},
 'HEMIPROP': {'Long Probability': 0.86, 'Short Probability': 0.52},
 'JSWENERGY': {'Long Probability': 0.86, 'Short Probability': 0.71},
 'M&MFIN': {'Long Probability': 0.86, 'Short Probability': 0.52}}

# <font color='Teal'>Live Data</font>


## Market Sentiment: TICK, TRIN (Arm's Index) & 52W high-Low touching shares


In [12]:
MS.get_live_sentiment()

{'Latest Updated on': '24/09/2021 12:34:59',
 'Up': 596,
 'Down': 898,
 'TICK': -302,
 'Volume Up': 1110.06,
 'Volume Down': 543.68,
 'TRIN': 0.325,
 '52W High': 67,
 '52W Low': 7}

## Indices Based
Buy or Sell stock based on which indices are performing highest in first 5 mins. Buy or sell stocks only for those indices which arevperforming the greatest

In [13]:
NSE.current_indices_status(7) # Live as of market is open. works post market also

,index,percentChange,key,variation
8,INDIA VIX,3.88,BROAD MARKET INDICES,0.64
26,NIFTY REALTY,2.61,SECTORAL INDICES,12.90
22,NIFTY METAL,-1.45,SECTORAL INDICES,-81.35
24,NIFTY PSU BANK,-1.35,SECTORAL INDICES,-32.50
27,NIFTY HEALTHCARE INDEX,-1.22,SECTORAL INDICES,-111.25
20,NIFTY IT,0.96,SECTORAL INDICES,352.75
46,NIFTY COMMODITIES,-0.92,THEMATIC INDICES,-51.70


In [15]:
NSE.open_nse_index('NIFTY REALTY')

,symbol,pChange,open,dayHigh,dayLow,lastPrice,Index
1,OBEROIRLTY,12.27,854.90,964.00,847.00,951.30,Nifty 200
2,DLF,2.92,404.40,418.30,397.70,413.60,Nifty 100
3,PHOENIXLTD,2.74,935.00,954.80,914.00,949.00,Nifty 500
0,NIFTY REALTY,2.61,496.55,510.05,492.25,506.80,Other
4,PRESTIGE,2.48,459.00,473.90,452.50,467.65,Nifty 200
10,HEMIPROP,-1.48,150.45,152.70,146.25,146.45,Nifty 500
5,SUNTECK,1.09,496.00,507.00,486.00,495.95,Nifty 500
6,SOBHA,0.73,819.00,840.00,791.20,816.55,Nifty 500
7,IBREALEST,0.45,154.70,161.00,154.20,154.85,Nifty 500
8,GODREJPROP,0.17,2199.85,2273.75,2172.25,2203.05,Nifty 200


# <font color='teal'>Risk Management</font>

In [16]:
# You have a total budget of 40K, and a risk apetite of rs 400 per trade
# You can set manual such as # entry = 3980.1, stop_loss=3780.5

In.get_particulars('ALKEM',40000,400,)

{'Buying Price': 3982.83,
 'Stop-Loss %': 3.99,
 'Target %': 7.98,
 'Quantity': 2.0,
 'Stop-Loss Price': 3823.82235,
 'Trigger Price': 4300.68,
 'Risk Per Share': 159.01,
 'Profit Per Share': 317.85,
 'Max loss on this config': 318.01,
 'Max Gain on this config': 635.71,
 'Price To Profit Ratio': 12.52,
 'Index': 'Nifty 100'}

In [18]:
r = requests.get('https://api.tickertape.in/product/tape')

In [19]:
r.json()

{'success': True,
 'data': [{'sid': '.NSEI', 'ticker': 'NIFTY 50'},
  {'sid': '.NSEBANK', 'ticker': 'NIFTY BANK'},
  {'sid': 'BJFN', 'ticker': 'BAJFINANCE'},
  {'sid': 'BRTI', 'ticker': 'BHARTIARTL'},
  {'sid': 'HDBK', 'ticker': 'HDFCBANK'},
  {'sid': 'HLL', 'ticker': 'HINDUNILVR'},
  {'sid': 'INGL', 'ticker': 'INDIGO'},
  {'sid': 'ITC', 'ticker': 'ITC'},
  {'sid': 'MRTI', 'ticker': 'MARUTI'},
  {'sid': 'RELI', 'ticker': 'RELIANCE'},
  {'sid': 'SBI', 'ticker': 'SBIN'},
  {'sid': 'TCS', 'ticker': 'TCS'}]}